# Human eval of SQuAD-like data

In [18]:
import json
import numpy as np
import random

In [19]:
REGION_CODE = 'ru'
K_TOP = 1

INPUT_FILE = '../data/\synthetic_google_top_n/%s/top_%d/train-synthetic.json' % (REGION_CODE, K_TOP)
OUTPUT_FILE = '../data/\synthetic_google_top_n/%s/top_%d/human-qa-evaluation.json' % (REGION_CODE, K_TOP)
DEFAULT_SEED = 42

np.random.seed(DEFAULT_SEED)
random.seed(DEFAULT_SEED)

In [20]:
with open(INPUT_FILE, 'r', encoding='utf-8') as fp:
    dataset = json.load(fp)

## Pick random sample

Picks some random items from the original dataset. There is a placeholder where a person has to write manually the answer.

In [21]:
N_SAMPLES = 50

In [22]:
n_items = len(dataset['data'])
print('Dataset size: %d' % n_items)

Dataset size: 2150


In [23]:
random_items = random.sample(range(n_items), N_SAMPLES)

In [24]:
subdataset = []
for i, item in enumerate(dataset['data']):
    if i not in random_items:
        continue
    item = item['paragraphs'][0]
    for qa_item in item['qas']:
        subdataset.append({
            'context': item['context'],
            'question': qa_item['question'],
            'computer_answers': list(set([x['text'] for x in qa_item['answers']])),
            'valid_exact_context': 0,
            'valid_enough_context': 0,
        })

In [25]:
with open(OUTPUT_FILE, 'w', encoding='utf-8') as fp:
    json.dump(subdataset, fp, ensure_ascii=False)

## Validate results

Once a human has written the answers, run this section: it reloads the sub-dataset and validates the answers.

In [27]:
with open(OUTPUT_FILE, 'r', encoding='utf-8') as fp:
    qadataset = json.load(fp)

In [28]:
n_items = len(qadataset)
qa_result = {
    'valid_exact_context': 0,
    'valid_enough_context': 0,
}

for item in qadataset:
    if item['valid_exact_context']:
        qa_result['valid_exact_context'] += 1
    if item['valid_enough_context']:
        qa_result['valid_enough_context'] += 1

In [29]:
print('QA size: %d' % n_items)
print('- Exact answers: %.2f%%' % (qa_result['valid_exact_context'] / n_items * 100))
print('- Enough info: %.2f%%' % (qa_result['valid_enough_context'] / n_items * 100))

QA size: 50
- Exact answers: 36.00%
- Enough info: 56.00%
